# Part II

## 2.1 Inventory routing model

Build and solve an inventory routing model in Julia in order to determine the optimal routes for the trucks,
and the optimal inventory levels for the DCs, assuming there is no uncertainty (so only use the means of the
salmon demand distribution). Report on the routes used and the inventory levels over the different days.
Hint: If your model is correct, you should get an optimal cost of 4881.57.

In [3]:
using JuMP
using Gurobi


#------
# DATA
#------

### SETS
vehicles = [1 2 3];
K = length(vehicles);
DCs = [ 1 2 3 4 5 6];
I = length(DCs);
days = [1 2 3 4 5 6 7 8]
T = length(days); #number of time periods

Capdc = [999999 502 488 220 486 742]; #inventory holding capacity at every node including depot
Captruck = 900; #capacity of each truck
h = 0.025; # inventory holding cost at customers
Inv_begin = [0 98.8 153.9 42.4 23.4 85.7]; # inventory at the nodes at the beginning of the period

shippingCost = [0 140 434 389 419 125
    140 0 300 455 400 97
    434 300 0 609 417 330
    389 455 609 0 256 358
    419 400 417 256 0 316
    125 97 330 358 316 0
]; # transporation cost including from depot (matrix flipped from original data)

demand = [0 0 0 0 0 0 0 0 
    75.9 75.9 75.9 75.9 75.9 85.7 85.7 75.9
    62.9 94.8 94.8 94.8 94.8 68.65 36.75 62.9
    67.2 67.2 67.2 67.2 67.2 67.15 67.15 67.2
    102.3 156.0 156.0 156.0 156.0 119.05 65.35 102.3
    107.4 130.1 130.1 130.1 130.1 65.15 42.45 107.4
]; #demand at all customers (rows) for every time period(columns)

model = Model(Gurobi.Optimizer);

@variable(model, x[i=1:I,j=1:I,k=1:K,t=1:T], Bin); #vehicle k goes from i to j in period t
@variable(model, y[i=1:I,k=1:K,t=1:T], Bin); #vehicle k visits node i in period t
@variable(model, z[i=2:I,k=1:K,t=1:T] >= 0); #load carried by vehicle k when arriving at node i in period t
@variable(model, p[i=1:I,t=1:T] >=0); # inventory at node i in time period t
#@variable(model, r[t=1:T] >=0); #supply available at depot (node 0) at time t
@variable(model, q[i=2:I,k=1:K,t=1:T] >=0); # Quantity delivered to customer i by vehicle k at start of period t

@objective(model, Min,sum(h*p[i,t] for i in 2:I, t in 1:T)+ sum(shippingCost[i,j]*x[i,j,k,t] for i in 1:I,j in 1:I,k in 1:K,t in 1:T));

#_______________ pure VRP

# Constraint 4: A customer got visit by not more than one truck in one period:
@constraint(model, [i=2:I,t=1:T], sum(y[i,k,t] for k in 1:K) <= 1);

# Constraint 5: Vehicle capacity
@constraint(model, [k=1:K,t=1:T], sum(demand[i,t]*y[i,k,t] for i in 1:I) <= Captruck);

# Constraint 5_1 : Inflow: if vehicle k visits node i, inflow of vehicle k should be 1, 0, otherwise
#                  Outflow: if vehicle k visits node i, outflow of vehicle k should be 1, 0, otherwise
#part 1
@constraint(model, [i=1:I,k=1:K,t=1:T], sum(x[i,j,k,t] for j in 1:I) == sum(x[j,i,k,t] for j in 1:I));
#part 2
@constraint(model, [i=1:I,k=1:K,t=1:T], sum(x[i,j,k,t] for j in 1:I) == y[i,k,t]);

# Constraint 6: Subtour elimination constraint : load should reduce while going from i to j
# @constraint(model, [i=2:I,j=2:J,k=1:K,t=1:T], z[i,k,t] - z[j,k,t]>= demand[i,t] -((1-x[i,j,k,t])*sum(demand[i,t] for i in 1:I)))
@constraint(model, [i=2:I,j=2:I,k=1:K,t=1:T], z[i,k,t] - z[j,k,t] >= demand[i,t] - ((1-x[i,j,k,t]) * sum(demand[a,t] for a in 1:I) ));
#not sure if D is sum demand from 1:I or 2:I

#_______________ Inventory optimization

# Constraint 0: Initializing the first set of inventories
#@constraint(model, [i= 1:I], I_time[i,1] == I_begin[i]); # I[i,0] from slide

# Constraint 1: Flow balance constraints at depot in each period t:


@constraint(model, [t=1:T-1], p[1,t] == p[1,t+1] + sum(q[i,k,t+1] for i in 2:I, k in 1:K));

# Constraint 2: Flow balance constraints at each customer i in each period t:

@constraint(model, [i=2:I,t=1:T], (t == 1 ? Inv_begin[i] : p[i,t-1]) + sum(q[i,k,t] for k in 1:K) == p[i,t] + demand[i,t])

# Constraint 3: Inventory capacity constraints at each customer i in each period t:

@constraint(model, [i=2:I,t=1:T], (t == 1 ? Inv_begin[i] : p[i,t-1]) + sum(q[i,k,t] for k in 1:K) <= Capdc[i]);

#_______________ Linking both parts

# Constraint 7: Linking the inventory part of the model to the VRP part:
    # Constraint 7a: Amount delivered to a customer cannot exceed the holding capacity of the customer

@constraint(model, [i=2:I,k=1:K,t=1:T], q[i,k,t] <= Capdc[i]*y[i,k,t]);

    # Constraint 7b: Inventory delivered to a customer cannot exceed the truck capacity

@constraint(model, [k=1:K,t=1:T], sum(q[i,k,t] for i in 2:I) <= Captruck*y[1,k,t]);

#-------
# SOLVE
#-------

optimize!(model);

println();

if termination_status(model) == MOI.OPTIMAL
    println("Optimal objective value = $(objective_value(model))")
    println("   ")
    println("----------------------")
    println("   ")

end

for t in 1:T
    println("Time period: $t")
        for k = 1:K
            println("Truck number: $k")
            for i = 1:I
                for j = 2:I
                    if (value(x[i,j,k,t]) == 1)
                        print("  ", i,"->",j," : ", round(value(q[j,k,t]),digits=2)," kg")
                        println("   ")
                    end
                end
            end
        end
    println("----------------------")
end


Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-18
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1183 rows, 1296 columns and 5101 nonzeros
Model fingerprint: 0x97dfa414
Variable types: 288 continuous, 1008 integer (1008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+02]
  Objective range  [3e-02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 9e+02]
Presolve removed 192 rows and 186 columns
Presolve time: 0.01s
Presolved: 991 rows, 1110 columns, 4399 nonzeros
Variable types: 273 continuous, 837 integer (837 binary)
Found heuristic solution: objective 7775.8112500

Root relaxation: objective 2.111587e+03, 385 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0

## 2.2 Fill rate
Build a simulation in Julia to estimate if the fill rate target can be met. Report on the fill rates for the different
DCs over the different days
Below is some code to help you get started. As you can see, the situation is simulated 10000 times and each
time actual demands are generated using the demand distribution for salmon (given by the matrices d and
std). The amount delivered to each DC on each day is obtained from the solution of the model from (1)
above.

In [6]:
using JuMP
using Gurobi
import Random


#------
# DATA
#------

### SETS
vehicles = [1 2 3];
K = length(vehicles);
DCs = [ 1 2 3 4 5 6];
I = length(DCs);
days = [1 2 3 4 5 6 7 8]
T = length(days); #number of time periods

Capdc = [999999 502 488 220 486 742]; #inventory holding capacity at every node including depot
Captruck = 900; #capacity of each truck
h = 0.025; # inventory holding cost at customers
Inv_begin = [0 98.8 153.9 42.4 23.4 85.7]; # inventory at the nodes at the beginning of the period

shippingCost = [0 140 434 389 419 125
    140 0 300 455 400 97
    434 300 0 609 417 330
    389 455 609 0 256 358
    419 400 417 256 0 316
    125 97 330 358 316 0
]; # transporation cost including from depot (matrix flipped from original data)

demand = [0 0 0 0 0 0 0 0 
    75.9 75.9 75.9 75.9 75.9 85.7 85.7 75.9
    62.9 94.8 94.8 94.8 94.8 68.65 36.75 62.9
    67.2 67.2 67.2 67.2 67.2 67.15 67.15 67.2
    102.3 156.0 156.0 156.0 156.0 119.05 65.35 102.3
    107.4 130.1 130.1 130.1 130.1 65.15 42.45 107.4
]; #demand at all customers (rows) for every time period(columns)



demand_std = [
    11.87434209	11.87434209	11.87434209	11.87434209	11.87434209	14.68843082	14.68843082	11.87434209
    10.90871211	13.89244399	13.89244399	13.89244399	13.89244399	14.14213562	11.22497216	10.90871211
    11.26942767	11.26942767	11.26942767	11.26942767	11.26942767	12.86468033	12.86468033	11.26942767
    13.19090596	16.03121954	16.03121954	16.03121954	16.03121954	15.04991694	11.97914855	13.19090596
    13.03840481	14	        14	        14	        14	        11.0792599	9.836157786	13.03840481
];

#demand_std = [
#    43 43 43 43 43 46.5 46.5 43
#    39 59 59 59 59 44 24 39
#    41 41 41 41 41 42 42 41
#    56 85 85 85 85 65 36 56
#    60 72 72 72 72 34.5 22.5 60
#]

model = Model(Gurobi.Optimizer);

@variable(model, x[i=1:I,j=1:I,k=1:K,t=1:T], Bin); #vehicle k goes from i to j in period t
@variable(model, y[i=1:I,k=1:K,t=1:T], Bin); #vehicle k visits node i in period t
@variable(model, z[i=2:I,k=1:K,t=1:T] >= 0); #load carried by vehicle k when arriving at node i in period t
@variable(model, p[i=1:I,t=1:T] >=0); # inventory at node i in time period t
#@variable(model, r[t=1:T] >=0); #supply available at depot (node 0) at time t
@variable(model, q[i=2:I,k=1:K,t=1:T] >=0); # Quantity delivered to customer i by vehicle k at start of period t

@objective(model, Min,sum(h*p[i,t] for i in 2:I, t in 1:T)+ sum(shippingCost[i,j]*x[i,j,k,t] for i in 1:I,j in 1:I,k in 1:K,t in 1:T));

#_______________ pure VRP

# Constraint 4: A customer got visit by not more than one truck in one period:
@constraint(model, [i=2:I,t=1:T], sum(y[i,k,t] for k in 1:K) <= 1);

# Constraint 5: Vehicle capacity
@constraint(model, [k=1:K,t=1:T], sum(demand[i,t]*y[i,k,t] for i in 1:I) <= Captruck);

# Constraint 5_1 : Inflow: if vehicle k visits node i, inflow of vehicle k should be 1, 0, otherwise
#                  Outflow: if vehicle k visits node i, outflow of vehicle k should be 1, 0, otherwise
#part 1
@constraint(model, [i=1:I,k=1:K,t=1:T], sum(x[i,j,k,t] for j in 1:I) == sum(x[j,i,k,t] for j in 1:I));
#part 2
@constraint(model, [i=1:I,k=1:K,t=1:T], sum(x[i,j,k,t] for j in 1:I) == y[i,k,t]);

# Constraint 6: Subtour elimination constraint : load should reduce while going from i to j
# @constraint(model, [i=2:I,j=2:J,k=1:K,t=1:T], z[i,k,t] - z[j,k,t]>= demand[i,t] -((1-x[i,j,k,t])*sum(demand[i,t] for i in 1:I)))
@constraint(model, [i=2:I,j=2:I,k=1:K,t=1:T], z[i,k,t] - z[j,k,t] >= demand[i,t] - ((1-x[i,j,k,t]) * sum(demand[a,t] for a in 1:I) ));
#not sure if D is sum demand from 1:I or 2:I

#_______________ Inventory optimization

# Constraint 0: Initializing the first set of inventories
#@constraint(model, [i= 1:I], I_time[i,1] == I_begin[i]); # I[i,0] from slide

# Constraint 1: Flow balance constraints at depot in each period t:


#@constraint(model, [t=1:T-1], p[1,t] == p[1,t+1] + sum(q[i,k,t+1] for i in 2:I, k in 1:K));

# Constraint 2: Flow balance constraints at each customer i in each period t:

@constraint(model, [i=2:I,t=1:T], (t == 1 ? Inv_begin[i] : p[i,t-1]) + sum(q[i,k,t] for k in 1:K) == p[i,t] + demand[i,t])

# Constraint 3: Inventory capacity constraints at each customer i in each period t:

@constraint(model, [i=2:I,t=1:T], (t == 1 ? Inv_begin[i] : p[i,t-1]) + sum(q[i,k,t] for k in 1:K) <= Capdc[i]);

#_______________ Linking both parts

# Constraint 7: Linking the inventory part of the model to the VRP part:
    # Constraint 7a: Amount delivered to a customer cannot exceed the holding capacity of the customer

@constraint(model, [i=2:I,k=1:K,t=1:T], q[i,k,t] <= Capdc[i]*y[i,k,t]);

    # Constraint 7b: Inventory delivered to a customer cannot exceed the truck capacity

@constraint(model, [k=1:K,t=1:T], sum(q[i,k,t] for i in 2:I) <= Captruck*y[1,k,t]);

#-------
# SOLVE
#-------

optimize!(model);

println();

if termination_status(model) == MOI.OPTIMAL
    println("Optimal objective value = $(objective_value(model))")
    println("   ")
    println("----------------------")
    println("   ")

end




fill_rates_count = zeros(I-1,T)
Random.seed!(0);
for r = 1:10000
    inv = Inv_begin[2:end]
    actual_demand = demand[2:end,:] + (randn(I-1,T).* demand_std)
    for t=1:T
        for i=1:I-1
            if actual_demand[i,t] == 0
                println(r, " ",t, " ",i)
            end
            delivered = 0
            for k=1:K
                delivered += value(q[i+1,k,t])
            end
            inv[i] = inv[i] + delivered - actual_demand[i,t]
            demand_met_from_stock =  max(actual_demand[i,t] + min(inv[i],0),0)
            fill_rate = demand_met_from_stock / actual_demand[i,t]
            if fill_rate>=0.95
                fill_rates_count[i,t] += 1
            end
            #println("DC: ",i)
            #println("Demand: ",actual_demand[i,t])
            #println(" | Inventory: ",actual_inventory[i,t])
            #println(fill_rate)
        end
    end 
end
fill_rates_count/10000

Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-18
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1176 rows, 1296 columns and 4982 nonzeros
Model fingerprint: 0xa9ca8870
Variable types: 288 continuous, 1008 integer (1008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+02]
  Objective range  [3e-02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 9e+02]
Presolve removed 185 rows and 186 columns
Presolve time: 0.05s
Presolved: 991 rows, 1110 columns, 4399 nonzeros
Variable types: 273 continuous, 837 integer (837 binary)
Found heuristic solution: objective 7775.8112500

Root relaxation: objective 2.111587e+03, 385 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0

5×8 Matrix{Float64}:
 1.0  1.0     1.0  0.9997  0.5613  1.0     0.9923  0.5434
 1.0  1.0     1.0  0.9999  0.5654  0.9988  0.9683  0.5354
 1.0  0.5799  1.0  0.9997  0.6079  1.0     0.9885  0.5421
 1.0  0.6491  1.0  1.0     0.5939  1.0     0.9953  0.5532
 1.0  1.0     1.0  1.0     1.0     1.0     0.9995  0.5575

## 2.3 Determine a safety factor
The fact that the simulation shows that the fill rate target is not met should not be surprising, as the inventory
routing model ignored the uncertainty in demand. Adapt the inventory routing model so that it includes some
safety stock. Instead of using the mean demand, use the mean demand plus the standard deviation multiplied
by a safety factor (use a common safety factor for all demands). What safety factor is needed to meet the fill
rate target? What is the effect on the total cost?

In [10]:
using JuMP
using Gurobi
import Random


#------
# DATA
#------

### SETS
vehicles = [1 2 3];
K = length(vehicles);
DCs = [ 1 2 3 4 5 6];
I = length(DCs);
days = [1 2 3 4 5 6 7 8]
T = length(days); #number of time periods

Capdc = [999999 502 488 220 486 742]; #inventory holding capacity at every node including depot
Captruck = 900; #capacity of each truck
h = 0.025; # inventory holding cost at customers
Inv_begin = [0 98.8 153.9 42.4 23.4 85.7]; # inventory at the nodes at the beginning of the period

shippingCost = [0 140 434 389 419 125
    140 0 300 455 400 97
    434 300 0 609 417 330
    389 455 609 0 256 358
    419 400 417 256 0 316
    125 97 330 358 316 0
]; # transporation cost including from depot (matrix flipped from original data)

demand = [0 0 0 0 0 0 0 0 
    75.9 75.9 75.9 75.9 75.9 85.7 85.7 75.9
    62.9 94.8 94.8 94.8 94.8 68.65 36.75 62.9
    67.2 67.2 67.2 67.2 67.2 67.15 67.15 67.2
    102.3 156.0 156.0 156.0 156.0 119.05 65.35 102.3
    107.4 130.1 130.1 130.1 130.1 65.15 42.45 107.4
]; #demand at all customers (rows) for every time period(columns)





demand_std = [0 0 0 0 0 0 0 0
    11.87434209	11.87434209	11.87434209	11.87434209	11.87434209	14.68843082	14.68843082	11.87434209
    10.90871211	13.89244399	13.89244399	13.89244399	13.89244399	14.14213562	11.22497216	10.90871211
    11.26942767	11.26942767	11.26942767	11.26942767	11.26942767	12.86468033	12.86468033	11.26942767
    13.19090596	16.03121954	16.03121954	16.03121954	16.03121954	15.04991694	11.97914855	13.19090596
    13.03840481	14	        14	        14	        14	        11.0792599	9.836157786	13.03840481
];

Random.seed!(0);
actual_demand = zeros(10000,I-1,T)
for r = 1:10000
    actual_demand[r,:,:] = demand[2:end,:] + (randn(I-1,T).* demand_std[2:end,:])
end

function optimize_vrpinv(za)
    model = Model(Gurobi.Optimizer);

    @variable(model, x[i=1:I,j=1:I,k=1:K,t=1:T], Bin); #vehicle k goes from i to j in period t
    @variable(model, y[i=1:I,k=1:K,t=1:T], Bin); #vehicle k visits node i in period t
    @variable(model, z[i=2:I,k=1:K,t=1:T] >= 0); #load carried by vehicle k when arriving at node i in period t
    @variable(model, p[i=1:I,t=1:T] >=0); # inventory at node i in time period t
    #@variable(model, r[t=1:T] >=0); #supply available at depot (node 0) at time t
    @variable(model, q[i=2:I,k=1:K,t=1:T] >=0); # Quantity delivered to customer i by vehicle k at start of period t
    #@variable(model, za >=0); #safety stock
    #@variable(model, f[r=1:10000,i=2:I,t=1:T], Bin);

    @objective(model, Min,sum(h*p[i,t] for i in 2:I, t in 1:T) + sum(shippingCost[i,j]*x[i,j,k,t] for i in 1:I,j in 1:I,k in 1:K,t in 1:T));

    #_______________ pure VRP

    # Constraint 4: A customer got visit by not more than one truck in one period:
    @constraint(model, [i=2:I,t=1:T], sum(y[i,k,t] for k in 1:K) <= 1);

    # Constraint 5: Vehicle capacity
    @constraint(model, [k=1:K,t=1:T], sum(demand[i,t]*y[i,k,t] for i in 1:I) <= Captruck);

    # Constraint 5_1 : Inflow: if vehicle k visits node i, inflow of vehicle k should be 1, 0, otherwise
    #                  Outflow: if vehicle k visits node i, outflow of vehicle k should be 1, 0, otherwise
    #part 1
    @constraint(model, [i=1:I,k=1:K,t=1:T], sum(x[i,j,k,t] for j in 1:I) == sum(x[j,i,k,t] for j in 1:I));
    #part 2
    @constraint(model, [i=1:I,k=1:K,t=1:T], sum(x[i,j,k,t] for j in 1:I) == y[i,k,t]);

    # Constraint 6: Subtour elimination constraint : load should reduce while going from i to j
    # @constraint(model, [i=2:I,j=2:J,k=1:K,t=1:T], z[i,k,t] - z[j,k,t]>= demand[i,t] -((1-x[i,j,k,t])*sum(demand[i,t] for i in 1:I)))
    @constraint(model, [i=2:I,j=2:I,k=1:K,t=1:T], z[i,k,t] - z[j,k,t] >= demand[i,t] - ((1-x[i,j,k,t]) * sum(demand[a,t] for a in 1:I) ));
    #not sure if D is sum demand from 1:I or 2:I

    #_______________ Inventory optimization

    # Constraint 0: Initializing the first set of inventories
    #@constraint(model, [i= 1:I], I_time[i,1] == I_begin[i]); # I[i,0] from slide

    # Constraint 1: Flow balance constraints at depot in each period t:


    @constraint(model, [t=1:T-1], p[1,t] == p[1,t+1] + sum(q[i,k,t+1] for i in 2:I, k in 1:K));

    # Constraint 2: Flow balance constraints at each customer i in each period t:

    @constraint(model, [i=2:I,t=1:T], (t == 1 ? Inv_begin[i] : p[i,t-1]) + sum(q[i,k,t] for k in 1:K) == p[i,t] + demand[i,t] + za*demand_std[i,t])

    # Constraint 3: Inventory capacity constraints at each customer i in each period t:

    @constraint(model, [i=2:I,t=1:T], (t == 1 ? Inv_begin[i] : p[i,t-1]) + sum(q[i,k,t] for k in 1:K) <= Capdc[i]);

    #_______________ Linking both parts

    # Constraint 7: Linking the inventory part of the model to the VRP part:
        # Constraint 7a: Amount delivered to a customer cannot exceed the holding capacity of the customer

    @constraint(model, [i=2:I,k=1:K,t=1:T], q[i,k,t] <= Capdc[i]*y[i,k,t]);

        # Constraint 7b: Inventory delivered to a customer cannot exceed the truck capacity

    @constraint(model, [k=1:K,t=1:T], sum(q[i,k,t] for i in 2:I) <= Captruck*y[1,k,t]);

    #-------
    # SOLVE
    #-------

    optimize!(model);

    println();
    #if termination_status(model) == MOI.OPTIMAL
    println("Optimal objective value = $(objective_value(model))")
    println("   ")
    println("----------------------")
    println("   ")
    #end

    fill_rates_count = zeros(I-1,T)
    Random.seed!(0);
    for r = 1:10000
        inv = Inv_begin[2:end]
        actual_demand = demand[2:end,:] + (randn(I-1,T).* demand_std[2:end,:])
        for t=1:T
            for i=1:I-1
                delivered = 0
                for k=1:K
                    delivered += value(q[i+1,k,t])
                end
                inv[i] = inv[i] + delivered - actual_demand[i,t]
                demand_met_from_stock =  max(actual_demand[i,t] + min(inv[i],0),0)
                fill_rate = demand_met_from_stock / actual_demand[i,t]
                if fill_rate>=0.95
                    fill_rates_count[i,t] += 1
                end
            end
        end 
    end
    return fill_rates_count/10000, objective_value(model)
end

za = [2.3263] #F^-1(0.99)
objectivevalues = []
global fill_rates = ones(I-1,T)
target = 0.99
step = -0.10


while all(fill_rates .> target)
    println("Iteration: ",length(za))
    println("--------------------------")
    println("Safety factor: ",za[end])
    global fill_rates,objectivevalue = optimize_vrpinv(za[end])
    append!(za,za[end]+step)
    append!(objectivevalues,objectivevalue)
end

println("------------------------------")
println("Incremental search finished. Best safety-factor found: ",round(za[end-1],digits=4)," optimal safety factor between $(round(za[end-1],digits=4)) and $(round(za[end],digits=4))")
println("Optimal objective value = $(objectivevalues[end-1])")

Iteration: 1
--------------------------
Safety factor: 2.3263
Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-18
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1183 rows, 1296 columns and 5101 nonzeros
Model fingerprint: 0x7e389e4e
Variable types: 288 continuous, 1008 integer (1008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+02]
  Objective range  [3e-02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 9e+02]
Presolve removed 193 rows and 186 columns
Presolve time: 0.05s
Presolved: 990 rows, 1110 columns, 4402 nonzeros
Variable types: 274 continuous, 836 integer (836 binary)

Root relaxation: objective 2.941497e+03, 483 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node